# CNN Class definition

In [2]:
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

/opt/conda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
class SimpleCNN(Chain):
    
    def __init__(self, input_channel, output_channel, filter_size, mid_units, n_units, n_label):
        super(SimpleCNN, self).__init__(
            conv1 = L.Convolution2D(input_channel, output_channel, (filter_size, filter_size)),
            l1    = L.Linear(mid_units, n_units),
            l2    = L.Linear(n_units,  n_label),
        )
    
    #Classifier
    def __call__(self, x):
        h1 = F.max_pooling_2d(F.relu(self.conv1(x)), (2,2))
        h2 = F.dropout(F.relu(self.l1(h1)))
        y = self.l2(h2)
        return y

#     def forward(self, x, t, train=True):
#         h1 = F.max_pooling_2d(F.relu(self.conv1(x)), 3)
#         h2 = F.dropout(F.relu(self.l1(h1)), train=train)
#         y = self.l2(h2)
#         return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

# Training Procedure definition

In [5]:
import numpy as np
from sklearn.cross_validation import train_test_split
from collections import defaultdict
import six
import sys
import chainer
import chainer.links as L
from chainer import optimizers, cuda, serializers
import chainer.functions as F
import argparse
from collections import namedtuple, defaultdict, deque

In [6]:
# load the dataset
train, test = chainer.datasets.get_mnist()

In [7]:
gpu = -1
n_epoch = 10
batchsize = 128
mnunits = 1690
nuits = 100

filter_size = 4
n_label = 10
input_channel = 1
output_channel = 10

model = L.Classifier(SimpleCNN(input_channel, output_channel, filter_size, mnunits, nuits, n_label))
# model = SimpleCNN(input_channel, output_channel, filter_size, mnunits, nuits, n_label)

xp = np
if gpu >= 0:
    cuda.check_cuda_available()
    cuda.get_device(gpu).use()
    model.to_gpu()
    xp = cuda.cupy

# Setup optimizer
optimizer = optimizers.AdaGrad()
optimizer.setup(model)

# Learning loop
stats = defaultdict(lambda: deque(maxlen=25))
for epoch in range(1, n_epoch + 1):

    print 'epoch', epoch, '/', n_epoch
    
    # training)
    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

    for itr, batch in enumerate(train_iter):
        
        xs = xp.array([datum[0][None, :].reshape(1,28,28) for datum in batch])
        ts = xp.array([datum[1] for datum in batch])

#         print(xs.shape)
#         print(ts.shape)
        
#         ys = model(xs)
        
#         loss = F.softmax_cross_entropy(ys, ts)
        
        optimizer.update(model, xs, ts)
        
        stats["loss"].append(float(loss.data))
#         stats["accuracy"].append(float((ys.data.argmax(1) == ys).sum() / batchsize))

        if itr % 300 == 0:
            print("; ".join("%s: %s" % (k, np.mean(vs)) for k, vs in stats.items()))

    sys.stdout.flush()

print 'save the model'
serializers.save_npz('./model/pn_classifier_cnn.model', model)
print 'save the optimizer'
serializers.save_npz('./model/pn_classifier_cnn.state', optimizer)

epoch 1 / 10


/opt/conda/lib/python2.7/site-packages/chainer/optimizers/ada_grad.py:50: RuntimeWarning: invalid value encountered in sqrt
  param.data -= lr * grad / (numpy.sqrt(h) + eps)


NameError: name 'loss' is not defined